In [35]:
%load_ext autoreload
%autoreload 2

from schema2db import validate
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np

In [12]:
inputfile = '../tests/testdata/testschema1.sql'
with open(inputfile, 'r') as f:
    inputstr = f.read()

In [14]:
components = inputstr.split(";")

In [15]:
components[0]

'create table prices (\n   itemid int(16) unsigned not null,\n   itemname varchar(20) not null,\n   price decimal(9,6) signed not null,\n   primary key (itemid)\n)'

In [16]:
table_str = components[0]

In [79]:
def get_table_name(words):
    if words[1].lower() != 'table'.lower():
        raise ValueError("You can only create tables, not {}".format(words[1]))
    else:
        return words[2]

def get_instruction(words):
    if words[0].lower() not in ['alter', 'create']:
        raise NotImplementedError("{} operation not supported".format(words[0]))
    else:
        return words[0].lower()

def extract_datatype(words):
    strtype = {}
    next_pos = 1
    strtype['type'] = (re.findall("[a-zA-Z]+", words[0]))[0].lower()
    strtype['args'] = re.findall("[0-9]+", words[0])
    if len(words) > 1 and words[1].lower() == 'signed':
        strtype['signed'] = True
        next_pos += 1
    elif len(words) > 1 and words[1].lower() == 'unsigned':
        strtype['signed'] = False
        next_pos += 1
    return strtype, words[next_pos:]

def extract_null(words):
    ifnull = None
    if 'not null' in " ".join([s for s in words]):
        ifnull = False
        nextpos = 2
    elif 'null' in " ".join([s for s in words]):
        ifnull = True
        nextpos = 1
    if len(words) > nextpos:
        return ifnull, words[1:]
    else:
        return ifnull, None
    

def parse_alter(sql_cmds):
    constraints = []
    pass

In [59]:
table = {}
table_strs = table_str.split("\n")
table_words = [s.split() for s in table_strs]

operation = get_instruction(table_words[0])

if operation == 'create':
    return parse_create(table_words[1:])
elif operation == 'alter':
    return parse_alter(table_words[1:])
else:
    raise NotImplementedError()

table['name'] = get_table_name(table_words[0])

table_words

SyntaxError: 'return' outside function (<ipython-input-59-6645ce91396b>, line 8)

In [60]:
table_words[1][1]

'int(16)'

In [83]:
def parse_create(c):
    cmds = []
    cmds = {'colname': c[0]}
    cmds['type'], cmd_remaining = extract_datatype(c[1:])
    cmds['null'], cmd_remaining = extract_null(cmd_remaining)
    return cmds

In [84]:
parse_create(table_words[1])

['int(16)', 'unsigned', 'not', 'null,']


{'colname': 'itemid',
 'type': {'type': 'int', 'args': ['16'], 'signed': False},
 'null': False}

In [50]:
" ".join(" not  null".split())

'not null'